In [1]:
from geography.FullProcess import FullProcess

FullProcess()

starting excel process for peter
gron/excel folder already exists


UnboundLocalError: local variable 'driver' referenced before assignment

# Tests

In [1]:
from geography.classes.UserClass import UserClass

In [2]:
from geography.classes.LoginClass import PasswordManager, WebDriverManager, Login

In [3]:
from geography.classes.NoLinkClass import NoLinkClass

In [4]:
from geography.classes.DownloadClass import Download

In [5]:
basin_code = "gron"
master_user = "peter"
download_type = "excel"

In [6]:
currentUser = UserClass(basin_code, master_user, download_type)

In [7]:
currentUser.getName() # just printing this line...

starting excel process for peter


In [8]:
paths = currentUser.getPath(download_type) # returns paths as dicts
paths

{'base_path': '/Users/pnadel01',
 'user_name': 'pnadel01',
 'geography_folder': './',
 'download_folder_temp': '/Users/pnadel01/Downloads/',
 'download_folder': './data/statusgron/excel/',
 'status_file': './data/status/excel/gron.csv'}

In [9]:
import os
for k, v in paths.items():
    print(f"{k}: ", os.path.exists(v)) # check if the path exists

base_path:  True
user_name:  False
geography_folder:  True
download_folder_temp:  True
download_folder:  True
status_file:  True


In [10]:
base_path = paths['base_path']
user_name = paths["user_name"]
geography_folder = paths["geography_folder"]
download_folder_temp = paths["download_folder_temp"]
download_folder = paths["download_folder"]
status_file = paths["status_file"]

In [11]:
if os.path.exists(download_folder):
    print(f"{basin_code}/{download_type} folder already exists")
else:
    os.makedirs(download_folder, exist_ok=True) # this isn't exactly right is it? - Yes, I added exists_ok=True to avoid errors if the folder already exists
    print(f"created folder {basin_code}/{download_type}")

gron/excel folder already exists


In [12]:
pm = PasswordManager()

In [13]:
if not pm.password:
    print("No password found, please enter your password")
    pm.get_password()
    print("Password saved successfully")

No password found, please enter your password
Password saved successfully


In [14]:
manager = WebDriverManager()

In [15]:
options = manager.setup_options() # not using this

In [16]:
driver = manager.start_driver()

In [17]:
password = pm.password
login = Login(user_name=user_name, password=password, driver_manager=manager, url=None)

In [18]:
login._init_login()

User is already logged in.


In [19]:
nlc = NoLinkClass(driver, basin_code, download_type, currentUser) 
# first main issue: Documents is hard coded, should be passed paths dict

In [21]:
nlc._search_process()

Navigate to Nexis Uni home page
Initializing search for gron
clicked search, on results page


In [23]:
download = Download(
    driver=driver,
    basin_code=basin_code,
    user_name=user_name,
    index=0,  
    login = login,
    nlc = nlc,
    download_folder = download_folder,
    download_folder_temp = download_folder_temp,
    status_file=status_file,
    finished=False,  
    url=None,  
    timeout=20 ,
    download_type='pdf' 
)

In [24]:
download.DownloadSetup() 

group duplicate results by moderate similarity
No popup found
Selected 'Date (oldest-newest)' option


In [25]:
download.reset()

logging out
deleting cookies before logging in again
User is already logged in.
already on Nexis Uni home page
Initializing search for gron
clicked search, on results page
group duplicate results by moderate similarity
No popup found
Selected 'Date (oldest-newest)' option


In [26]:
open_download_options = "//button[@class='has_tooltip' and @data-action='downloadopt']"
download._click_from_xpath(open_download_options)

In [ ]:
result_range_filed_xpath = "//input[@id='SelectedRange']"
download._send_keys_from_xpath(result_range_filed_xpath, "501-1000") # hwo to know when end

In [30]:
download_button = "//button[@type='submit' and @class='button primary' and @data-action='download']"
download._click_from_xpath(download_button)

In [ ]:
# move file to data folder
# downloads folder -> data folder

In [ ]:
download.reset()

In [ ]:
# get full coutn from site
full_count = 14_031 + 500
ranges = []

for i in range(1, full_count, 500):
    if i != 1:
        i -= 1
    ranges.append(f"{i}-{i+499}")

ranges[:3]

['1-500', '500-999', '1000-1499']

In [41]:
tracking = {r:False for r in ranges}

In [46]:
import pandas as pd
pd.DataFrame.from_dict(tracking, orient='index').reset_index().rename(columns={"index":"range", 0:"downloaded"})

range  downloaded
0         1-500       False
1       500-999       False
2     1000-1499       False
3     1500-1999       False
4     2000-2499       False
5     2500-2999       False
6     3000-3499       False
7     3500-3999       False
8     4000-4499       False
9     4500-4999       False
10    5000-5499       False
11    5500-5999       False
12    6000-6499       False
13    6500-6999       False
14    7000-7499       False
15    7500-7999       False
16    8000-8499       False
17    8500-8999       False
18    9000-9499       False
19    9500-9999       False
20  10000-10499       False
21  10500-10999       False
22  11000-11499       False
23  11500-11999       False
24  12000-12499       False
25  12500-12999       False
26  13000-13499       False
27  13500-13999       False
28  14000-14499       False
29  14500-14999       False

In [ ]:
tracking = {r:False for r in ranges}

download.DownloadSetup()
for r in ranges:
    # clicking on download button
    open_download_options = "//button[@class='has_tooltip' and @data-action='downloadopt']"
    download._click_from_xpath(open_download_options)

    # ender range
    result_range_filed_xpath = "//input[@id='SelectedRange']"
    download._send_keys_from_xpath(result_range_filed_xpath, r)

    # click on download
    download_button = "//button[@type='submit' and @class='button primary' and @data-action='download']"
    download._click_from_xpath(download_button)

    # move the files
    ## rename to result page

    tracking[r] = True
    tracked = pd.DataFrame.from_dict(tracking, orient='index').reset_index().rename(columns={"index":"range", 0:"downloaded"})
    tracked.to_csv(f'{basin_code}_tracker.csv', index=False)
    download.reset()

In [ ]:
# convert tracker back to tracking
tracked = pd.read_csv(f'{basin_code}_tracker.csv')
tracking = {r:True for r in tracked['range']}

In [22]:
download.main(index=0, basin_code = basin_code)

Download for gron
group duplicate results by moderate similarity
No popup found
Selected 'Date (oldest-newest)' option
Proceeding to download basin gron row 298
Setting the date range from 05/31/2015 to 06/06/2015


KeyboardInterrupt: 

# Functional

In [1]:
from geography.classes.UserClass import UserClass
from geography.classes.LoginClass import PasswordManager, WebDriverManager, Login
from geography.classes.NoLinkClass import NoLinkClass
from geography.classes.DownloadClass import Download

def get_user(basin, uname, dload_type):
    basin_code = basin
    master_user = uname
    download_type = dload_type

    currentUser = UserClass(basin_code, master_user, download_type)
    paths = currentUser.getPath(download_type)
    return paths, currentUser

def full_process(current_user):
    if os.path.exists(current_user.download_folder):
        print(f"{currentUser.basin}/{download_type} folder already exists")
    else:
        os.makedirs(current_user.download_folder, exist_ok=True)
        print(f"created folder {currentUser.basin}/{download_type}")
    
    pm = PasswordManager()
    if not pm.password:
        print("No password found, please enter your password")
        password = pm.get_password()
        print("Password saved successfully")
    
    manager = WebDriverManager()
    driver = manager.start_driver()

    login = Login(user_name=current_user.currentUser, password=password, driver_manager=manager, url=None)
    login._init_login()

    nlc = NoLinkClass(driver, current_user.basin, download_type, current_user)
    nlc._search_process()

    download = Download(
        driver=driver,
        basin_code=currentUser.basin,
        user_name=current_user.download_folder,
        index=0,
        login=login,
        nlc=nlc,
        download_folder=current_user.download_folder,
        download_folder_temp=current_user.download_folder_temp,
        status_file=current_user.status_file,
        finished=False,
        url=None,
        timeout=20
    )
    download.main(index=0, basin_code=currentUser.basin)

In [2]:
import pandas as pd
import os

basin_code = "gron"
master_user = "pnadel01"
download_type = "excel"

paths, current_user = get_user(basin_code, master_user, download_type)
status_file = paths["status_file"]

status_data = pd.read_csv(status_file, index_col=0)
row_index = 0
while row_index < len(status_data):
    # Check if all rows are finished
    if (status_data['finished'] == 1).all():
        print(f"All rows for {basin_code} are downloaded!")
        break
    else:
        full_process(current_user)

gron/excel folder already exists
No password found, please enter your password
Password saved successfully
User is already logged in.
already on Nexis Uni home page
Initializing search for gron
clicked search, on results page


NameError: name 'user_name' is not defined

# Chrome driver download

In [1]:
import os
import platform
import subprocess
import sys
import json
import shutil
from pathlib import Path
import requests
from zipfile import ZipFile
from typing import Optional, Tuple

In [2]:
system = platform.system()
system

'Darwin'

In [3]:
try:
    if system == "Darwin":  # macOS
        process = subprocess.Popen(
            ['/Applications/Google Chrome.app/Contents/MacOS/Google Chrome', '--version'],
            stdout=subprocess.PIPE
        )
        version = process.communicate()[0].decode('UTF-8').replace('Google Chrome ', '').strip()
    elif system == "Windows":
        process = subprocess.Popen(
            ['reg', 'query', 'HKEY_CURRENT_USER\\Software\\Google\\Chrome\\BLBeacon', '/v', 'version'],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
        output = process.communicate()[0].decode('UTF-8')
        version = output.strip().split()[-1]
    else:
        print("Unsupported operating system")
        
    print("Version number: ", version.split('.')[0])  # Return major version number
except Exception as e:
    print(f"Error getting Chrome version: {e}")

Version number:  130


In [4]:
machine = platform.machine().lower()
machine

'arm64'

In [5]:
if system == "Darwin":  # macOS
    platform_name = "mac"
    if machine == "x86_64":
        arch = "x64"
    elif machine == "arm64":
        arch = "arm64"
    else:
        raise SystemError(f"Unsupported Mac architecture: {machine}")
elif system == "Windows":
    platform_name = "win"
    if machine == "amd64" or machine == "x86_64":
        arch = "x64"
    else:
        raise SystemError(f"Unsupported Windows architecture: {machine}")
else:
    raise SystemError(f"Unsupported operating system: {system}")
print(f"Platform: {platform_name}, Architecture: {arch}")

Platform: mac, Architecture: arm64


In [6]:
base_url = "https://googlechromelabs.github.io/chrome-for-testing"
known_versions_url = f"{base_url}/known-good-versions-with-downloads.json"

In [7]:
response = requests.get(known_versions_url)
response.raise_for_status()
versions_data = response.json()
versions_data

{'timestamp': '2024-11-08T10:09:43.145Z',
 'versions': [{'version': '113.0.5672.0',
   'revision': '1121455',
   'downloads': {'chrome': [{'platform': 'linux64',
      'url': 'https://storage.googleapis.com/chrome-for-testing-public/113.0.5672.0/linux64/chrome-linux64.zip'},
     {'platform': 'mac-arm64',
      'url': 'https://storage.googleapis.com/chrome-for-testing-public/113.0.5672.0/mac-arm64/chrome-mac-arm64.zip'},
     {'platform': 'mac-x64',
      'url': 'https://storage.googleapis.com/chrome-for-testing-public/113.0.5672.0/mac-x64/chrome-mac-x64.zip'},
     {'platform': 'win32',
      'url': 'https://storage.googleapis.com/chrome-for-testing-public/113.0.5672.0/win32/chrome-win32.zip'},
     {'platform': 'win64',
      'url': 'https://storage.googleapis.com/chrome-for-testing-public/113.0.5672.0/win64/chrome-win64.zip'}]}},
  {'version': '113.0.5672.35',
   'revision': '1121455',
   'downloads': {'chrome': [{'platform': 'linux64',
      'url': 'https://storage.googleapis.com/c

In [21]:
if platform_name == "win":
    platform_download_name = f"{platform_name}64"
else:  # mac
    platform_download_name = f"{platform_name}-{arch}"
platform_download_name

'mac-arm64'

In [24]:
chrome_major_version = version.split('.')[0]

matching_versions = [
    v for v in versions_data['versions']
    if v['version'].split('.')[0] == chrome_major_version
    and 'chromedriver' in v['downloads']
    and any(d['platform'] == platform_download_name for d in v['downloads']['chromedriver'])
]
matching_version = sorted(matching_versions, key=lambda x: [int(i) for i in x['version'].split('.')])[-1]
matching_version

{'version': '130.0.6723.116',
 'revision': '1356013',
 'downloads': {'chrome': [{'platform': 'linux64',
    'url': 'https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/linux64/chrome-linux64.zip'},
   {'platform': 'mac-arm64',
    'url': 'https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/mac-arm64/chrome-mac-arm64.zip'},
   {'platform': 'mac-x64',
    'url': 'https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/mac-x64/chrome-mac-x64.zip'},
   {'platform': 'win32',
    'url': 'https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/win32/chrome-win32.zip'},
   {'platform': 'win64',
    'url': 'https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/win64/chrome-win64.zip'}],
  'chromedriver': [{'platform': 'linux64',
    'url': 'https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/linux64/chromedriver-linux64.zip'},
   {'platform': 'mac-arm64',
    'url': 'https://storage.goo

In [26]:
download_info = next(
            (d for d in matching_version['downloads']['chromedriver'] if d['platform'] == platform_download_name),
            None
        )
download_info

{'platform': 'mac-arm64',
 'url': 'https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/mac-arm64/chromedriver-mac-arm64.zip'}

In [27]:
download_url = download_info['url']
download_url

'https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/mac-arm64/chromedriver-mac-arm64.zip'

In [28]:
driver_dir = Path("./chromedriver")
driver_dir.mkdir(exist_ok=True)

In [29]:
response = requests.get(download_url)
response.raise_for_status()

In [30]:
zip_path = driver_dir / "chromedriver.zip"

In [31]:
with open(zip_path, 'wb') as f:
    f.write(response.content)

In [32]:
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(driver_dir)

In [33]:
chromedriver_name = "chromedriver.exe" if platform_name == "win" else "chromedriver"
chromedriver_name

'chromedriver'

In [34]:
extracted_driver = next(driver_dir.rglob(chromedriver_name))
extracted_driver

PosixPath('chromedriver/chromedriver-mac-arm64/chromedriver')

In [35]:
final_driver_path = driver_dir / chromedriver_name
final_driver_path

PosixPath('chromedriver/chromedriver')

In [36]:
shutil.move(str(extracted_driver), str(final_driver_path))

'chromedriver/chromedriver'

In [37]:
zip_path.unlink()
for item in driver_dir.iterdir():
    if item.is_dir():
        shutil.rmtree(item)

In [38]:
if platform_name != "win":
    os.chmod(final_driver_path, 0o755)